In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Load in the stock data and create some basic features
data = pd.read_csv("../data/Coinbase_BTCUSD_dailydata.csv")

# Generate some fake columns just for formatting practice
data['net_worth'] = 100
data['num_stocks'] = 20

data.sort_values(by=["date"], inplace=True)
data.reset_index(drop=True, inplace=True)

In [3]:
# Create the features for the RNN
import tensorflow as tf
x = []
for i in range(5, len(data)):
    tensor_list = [
        data["unix"].iloc[i-5:i].values,#.reshape(1,-1),
        data["low"].iloc[i-5:i].values,#.reshape(1,-1),
        data["high"].iloc[i-5:i].values,#.reshape(1,-1),
        data["open"].iloc[i-5:i].values,#.reshape(1,-1),
        data["close"].iloc[i-5:i].values,#.reshape(1,-1),
        data["volume"].iloc[i-5:i].values,#.reshape(1,-1),
        data['net_worth'].iloc[i-1:i].values,#.reshape(1,-1),
        data['num_stocks'].iloc[i-1:i].values,#.reshape(1,-1)
    ]
    x.append(tensor_list)
    
y = np.ones(len(x))

In [6]:
ragged_tensor

<tf.RaggedTensor [[[1619308800.0, 1619395200.0, 1619481600.0, 1619568000.0, 1619654400.0],
  [47044.01, 48817.62, 53321.0, 53887.0, 52369.61],
  [50591.38, 54400.0, 55509.39, 56476.17, 55226.86], ...,
  [14633.512, 18005.225, 13957.087, 16484.336, 14592.316], [100.0],
  [20.0]],
 [[1619395200.0, 1619481600.0, 1619568000.0, 1619654400.0, 1619740800.0],
  [48817.62, 53321.0, 53887.0, 52369.61, 53068.43],
  [54400.0, 55509.39, 56476.17, 55226.86, 58075.01], ...,
  [18005.225, 13957.087, 16484.336, 14592.316, 16536.332], [100.0],
  [20.0]],
 [[1619481600.0, 1619568000.0, 1619654400.0, 1619740800.0, 1619827200.0],
  [53321.0, 53887.0, 52369.61, 53068.43, 57050.94],
  [55509.39, 56476.17, 55226.86, 58075.01, 58550.0], ...,
  [13957.087, 16484.336, 14592.316, 16536.332, 10787.05], [100.0], [20.0]],
 ...,
 [[1644537600.0, 1644624000.0, 1644710400.0, 1644796800.0, 1644883200.0],
  [42000.0, 41750.0, 41885.61, 41570.0, 42433.28],
  [43962.13, 43050.0, 42781.96, 42876.15, 44775.96], ...,
  [19367

In [5]:
ragged_tensor = tf.ragged.constant(x)

Metal device set to: Apple M1


2022-03-28 16:41:27.895150: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-28 16:41:27.895332: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [59]:
batch_size = 10

dataset = tf.data.Dataset.from_tensor_slices((ragged_tensor, y))
dataset = dataset.shuffle(10000, reshuffle_each_iteration=True)

# Split into train and test
dataset_train = dataset.take(100)
dataset_test = dataset.skip(100)

dataset_train = dataset_train.batch(batch_size).prefetch(100)
dataset_test = dataset_test.batch(batch_size).prefetch(100)

In [52]:
# Design the model
from tensorflow.keras import layers

# For ragged tensor , get maximum sequence length
max_seq = ragged_tensor.bounding_shape()[-1]

mdl = tf.keras.Sequential([
    # Input Layer with shape = [Any,  maximum sequence length]                      
    tf.keras.layers.Input(shape=[None, max_seq], dtype=tf.float32, ragged=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='softmax')
])

# CategoricalCrossentropy
mdl.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

mdl.summary()
history = mdl.fit(ragged_tensor, y, epochs=10)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 64)                17920     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 17,985
Trainable params: 17,985
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10


2022-03-25 12:29:27.974007: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-25 12:29:28.082830: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-25 12:29:28.136536: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 1s 15ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/10
10/10 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
10/10 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
10/10 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
10/10 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
10/10 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
10/10 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
10/10 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
10/10 [==============================] - 0s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
10/10 [==============================] - 0s 10ms/step - loss: 0.0000e+00